In [1]:
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import onnx
from onnx_tf.backend import prepare
import h5py
plt.style.use("ggplot")

/opt/conda/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
## Load datasets
combined_data = h5py.File("h5_datasets/combined_CT_datasets.h5", "r")

## Get train and test datasets
label_y = np.array(combined_data["label_test"])
y = np.array(combined_data["y"])

# predict using ONXX model trained remotely

#loaded_model = onnx.load("galaxy_trained_model/galaxy_trained_model_2.onnx") #onnx.load("onnx_loaded_model1/ct_seg_model.onnx")
# https://usegalaxy.eu/api/histories/d83af45c42c55bdb/contents/11ac94870d0bb33aa573fa238dbeaacd/display
## download data from Galaxy history

#path_trained_model = "https://usegalaxy.eu/api/histories/d83af45c42c55bdb/contents/11ac94870d0bb33aa573fa238dbeaacd/display"
#if not os.path.exists('galaxy_trained_model'):
#    os.makedirs('galaxy_trained_model')

# Write to an onnx file
# Get trained model
#r = requests.get(path_tm, allow_redirects=True)
#open('galaxy_trained_model/galaxy_trained_model_2.onnx', 'wb').write(r.content)
#loaded_model = onnx.load("galaxy_trained_model/galaxy_trained_model_2.onnx")

# predict using ONXX trained model
loaded_model = onnx.load("onnx_loaded_model/ct_seg_model.onnx")

tf_loaded_model = prepare(loaded_model)
prediction_onnx = tf_loaded_model.run(y, training=False)
prediction_onnx = np.array(prediction_onnx).squeeze()

onnx_preds_test_t1 = (prediction_onnx > 0.1).astype(np.uint8)
onnx_preds_test_t2 = (prediction_onnx > 0.3).astype(np.uint8)

In [3]:
## Plot predicted masks
no_pred = 5
def plot_sample(X, y, binary_preds1, binary_preds2):
    fig, axs = plt.subplots(no_pred, 4, figsize=(80, 80),sharex='all')
    rand_indx = np.random.randint(1, y.shape[0], no_pred)
    i = 0
    for ix in rand_indx:
        l = 1
        axs[i,0].imshow(X[ix, ..., 0], cmap='gray')
        axs[0,0].set_title('Original CT Images',fontweight="bold", size=40)

        axs[i,1].imshow(y[ix,:,:,l].squeeze(),cmap='gray')
        axs[0,1].set_title('Ground-Truth Mask',fontweight="bold", size=40)

        axs[i,2].imshow(binary_preds1[ix,:,:,l].squeeze(), cmap='gray', vmin=0, vmax=1)
        axs[0,2].set_title('Predicted Mask \n with BCE loss',fontweight="bold", size=40)
      
        axs[i,3].imshow(binary_preds2[ix,:,:,l].squeeze(), cmap='gray', vmin=0, vmax=1)
        axs[0,3].set_title('Predicted Masks \n with BCE+TV Loss',fontweight="bold", size=40)
        
        i += 1
      
        plt.rcParams["axes.grid"] = False
        plt.subplots_adjust(wspace=-0.5, hspace=0.1)

    plt.savefig("true_pred_dataset2.png")

In [ ]:
plot_sample(y, label_y, onnx_preds_test_t1, onnx_preds_test_t2)

In [ ]:
# Release GPU memory
gpu = tf.config.list_physical_devices('GPU')
if len(gpu) > 0:
    from numba import cuda
    cuda.select_device(0)
    cuda.close()